In [1]:
# Changing the working directory
import os
os.chdir('../')

In [2]:
from utils import *

In [3]:
# Pulling the students level descriptions
students = sky.getAdvancedList(73113)[
        ['user_id', 'level_description', 'grade_level', 'advisor_id']
    ].astype(
        {'user_id':'int64'
    })

In [4]:
# Pulling advisors for Tableau sorting
advisors = getAdvisors()

advisors.Section = advisors.Section.str.replace(' ', '')

In [5]:
semesterGrades = getSemesterGrades()

In [6]:
# Joining data
fullSemesterGrades = semesterGrades.merge(students, 'inner').merge(advisors, 'inner')

# Being a good boy
fullSemesterGrades.reset_index(inplace = True, drop = True)
fullSemesterGrades.drop_duplicates(inplace=True)

# Removing pesky Band/Chorus
cleanGrades = fullSemesterGrades[fullSemesterGrades.course_code != "ARTS0610"].copy()

In [7]:
cleanGrades.columns

Index(['user_id', 'firstname', 'lastname', 'course_title', 'course_code',
       'term_name', 'block', 'teacher_first', 'teacher_last',
       'department_name', 'advisor_id', 'section_code', 'grade', 'grade_plan',
       'comment', 'level_description', 'grade_level', 'advisor_first',
       'advisor_last', 'advisor_email', 'Section'],
      dtype='object', name='name')

In [8]:
wideGrades = cleanGrades.pivot(
        index=[
            'user_id', 'firstname', 'lastname',
            'course_title',  'teacher_last', 'section_code', 'grade_level',
            'level_description', 'advisor_id', 'advisor_last', 'advisor_email',
            'Section', 'comment', 'block'
        ],
        columns='grade_plan',
        values='grade'
    )

# Being a good boy
wideGrades.fillna("", inplace = True)
wideGrades.reset_index(inplace=True)
wideGrades.drop('section_code', axis=1, inplace=True)
wideGrades = wideGrades.assign(name = wideGrades.lastname + ", " + wideGrades.firstname).copy()

In [9]:
wideGrades = wideGrades.astype({
    "advisor_id":"category",
    'level_description':'category',
    'user_id':'category'
    }).rename(columns={
        'grade_level': 'Grade',
        'name':'Name',
        'advisor_last': 'Advisor',
        'course_title':'Course',
        'teacher_last': "Teacher"
})

In [10]:
levels = ['Middle School', 'Upper School']
advisors =  wideGrades.advisor_id.cat.categories

In [11]:
wideGrades.columns

Index(['user_id', 'firstname', 'lastname', 'Course', 'Teacher', 'Grade',
       'level_description', 'advisor_id', 'Advisor', 'advisor_email',
       'Section', 'comment', 'block', '', 'S1 Final Assessment',
       'S1 Pre-Assessment', 'Sem 1', 'Name'],
      dtype='object', name='grade_plan')

In [12]:
# data = wideGrades.loc[wideGrades.advisor_id == advisors[0], 
#                ['grade_level', 'user_id', 'course_title', 'teacher_last', 'S1 Pre-Assessment',
#                'S1 Final Assessment', 'Sem 1', 'comment']
#               ]

In [13]:
# advisor = wideGrades.loc[wideGrades.advisor_id == advisors[0], 
#                ['advisor_id', 'advisor_last', 'Section', 'advisor_email']
#               ].drop_duplicates()

In [14]:
ms = wideGrades.loc[wideGrades.level_description == 'Middle School',
               ['Grade', 'Name',  'Advisor',
                'Course', 'Teacher', 'S1 Pre-Assessment',
               'S1 Final Assessment', 'Sem 1']
              ].sort_values(['Grade', 'Advisor', 'Name'])

ms.fillna("", inplace = True)

In [15]:
updateSpreadsheet(
        ms, 
        sheet_id = '1juxSoJZjzoxR-i_QX-NFgyJvQr78j-IfkEriBoHUa5Y'
    )

In [16]:
us = wideGrades.loc[wideGrades.level_description == 'Upper School',
               ['Grade', 'Name',  'Advisor',
                'Course', 'Teacher', 'S1 Pre-Assessment',
               'S1 Final Assessment', 'Sem 1']
              ].sort_values(['Grade', 'Name'])

us.fillna("", inplace = True)

In [17]:
updateSpreadsheet(
        us, 
        sheet_id = '10oUvyQPKu5v_PSbsl1J0cKzsQEWIsSGZI5SsxkmfuKI'
    )